In [1]:
import pandas as pd
import numpy as np

In [2]:
soubor = pd.read_csv('clean_data_years_filter.csv',  encoding='utf-8')
soubor.head(2)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,2011-01-01,LEKAR,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,10.0,82.83,828.3,33.3333,333.0,F
1,2011-01-01,LEKARNA,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,71070.0,82.83,5886728.1,33.3333,2368998.0,F


###### "clean_data_years_filter.csv" obsahuje data, kde:

###### "Typ odběratele":
'LEKARNA', 'VETERINARNI_LEKAR', 'PRODEJCE_VLP','HYGIENICKA_STANICE', 'ZDRAVOTNICKE_ZARIZENI', 'LEKAR', 'OSOBA_POSKYTUJICI_ZDRAVOTNI_PECI'

###### Abychom nic nepokazily, vytvorime novou tabulku "soubor1" a přejmenujeme poslední sloupec

In [3]:
soubor1 = soubor.loc[:,['Období', 'ATC7', 'Počet denních definovaných dávek/balení celkem']]

In [4]:
soubor1 = soubor1.rename(index=str, columns={'Počet denních definovaných dávek/balení celkem': 'DDD_total'})
soubor1.head()

,Období,ATC7,DDD_total
0,2011-01-01,M01AE01,333.0
1,2011-01-01,M01AE01,2368998.0
2,2011-01-01,M01AE01,-45533.0
3,2011-01-01,M01AE01,600.0
4,2011-01-01,M01AE01,20.0


### Jako další krok bylo třeba seskupit všechny léky obsahující stejnou molekulu ATC. 
Stejné ATC totiž může být základem pro různé léky. Použily jsme metodu groupby, pomocí které jsme vytvořily součet napříč jednotlivými ATC za každý měsíc. Protože k tomuto výpočtu z původních patnácti sloupců v tabulce nám vystačí tři, jsme do syntaxe zapsaly jen “Období”, “ATC” a “DDD_total”. 

### Zkusime spojit jednotlive ATC do vyssi urovni molekuly. 

Proto napred socteme vsechny ATC, nazev kterych se zacina s R05:

Jsou:  R05','R05C','R05CA','R05CA01','R05CA02', 'R05CA03', 'R05CA04', 'R05CA05', 'R05CA06', 'R05CA07', 'R05CA08', 'R05CA09', 'R05CA10', 'R05CA11', 'R05CA12', 'R05CA13', 'R05CB', 'R05CB01', 'R05CB02', 'R05CB03', 'R05CB04', 'R05CB05', 'R05CB06', 'R05CB07', 'R05CB08', 'R05CB09', 'R05CB10', 'R05CB11', 'R05CB13', 'R05CB14', 'R05CB15', 'R05CB16', 'R05D', 'R05DA', 'R05DA01', 'R05DA03', 'R05DA04', 'R05DA05', 'R05DA06', 'R05DA07', 'R05DA08', 'R05DA09', 'R05DA10', 'R05DA11', 'R05DA12', 'R05DA20', 'R05DB', 'R05DB01', 'R05DB02', 'R05DB03', 'R05DB04', 'R05DB05', 'R05DB07', 'R05DB09', 'R05DB10', 'R05DB11', 'R05DB12', 'R05DB13', 'R05DB14', 'R05DB15', 'R05DB16', 'R05DB17', 'R05DB18', 'R05DB19', 'R05DB20','R05DB21', 'R05DB22', 'R05DB23', 'R05DB24', 'R05DB25', 'R05DB26', 'R05DB27', 'R05DB28', 'R05F', 'R05FB','R05FB01', 'R05FB02', 'R05X'







In [5]:
soubor_R05 = soubor1[soubor1['ATC7'].str.startswith('R05')]

In [6]:
soubor_R05.tail(3)

,Období,ATC7,DDD_total
87763,2018-03-01,R05X,NaN
87764,2018-03-01,R05X,NaN
87765,2018-03-01,R05X,NaN


In [7]:
leky_R05 = soubor_R05.groupby(['Období']).agg({'Období':'first', 'DDD_total':'sum'})
leky_R05.tail()

,Období,DDD_total
Období,,
2017-11-01,2017-11-01,5.855393e+06
2017-12-01,2017-12-01,6.414107e+06
2018-01-01,2018-01-01,7.562418e+06
2018-02-01,2018-02-01,1.114885e+07
2018-03-01,2018-03-01,8.664688e+06


In [8]:
# Exportujeme
leky_R05.to_csv('leky/leky_R05.csv', encoding = 'utf-8')

In [9]:
leky_R05['Období'] = pd.to_datetime(leky_R05['Období'])

In [10]:
leky_R05['year'] = leky_R05['Období'].dt.year.astype(np.int)

In [11]:
leky_R05['month'] = leky_R05['Období'].dt.month.astype(np.int)

In [12]:
leky_R05['quater'] = leky_R05['Období'].dt.quarter.astype(np.int)
#DatetimeIndex.quarter

#### Pro statistický model jsme potřebovaly vytvořit “trend”. Náš nejstarší rok je označen nulou a každý další je o jednotku vyšší.

In [13]:
leky_R05['trend'] = leky_R05['year']-2011

In [14]:
leky_R05.tail(3)

,Období,DDD_total,year,month,quater,trend
Období,,,,,,
2018-01-01,2018-01-01,7.562418e+06,2018,1,1,7
2018-02-01,2018-02-01,1.114885e+07,2018,2,1,7
2018-03-01,2018-03-01,8.664688e+06,2018,3,1,7


In [15]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\Katerina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Statistický model
Pro použití statistického modelu jsme potřebovaly vytvořit další sloupce pro rok, měsíc a kvartál pomocí metody knihovny pandas “.to_datetime”.

Program vnímá čísla měsíců (1–12) a kvartálů (1–4) jako hodnoty, což se nedá využít pro náš stats model. Aby program nevnímal číslo měsíců a kvartálů jako hodnotu, je třeba použít binární systém jedniček a nul.

### Proto použijeme metodu “.get_dummies” na sloupce “month” a “quater”.

In [16]:
Q01 = pd.get_dummies(leky_R05['quater'], prefix='quater')
Q01.head()

,quater_1,quater_2,quater_3,quater_4
Období,,,,
2011-01-01,1,0,0,0
2011-02-01,1,0,0,0
2011-03-01,1,0,0,0
2011-04-01,0,1,0,0
2011-05-01,0,1,0,0


In [17]:
M01 = pd.get_dummies(leky_R05['month'], prefix='month')
M01.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Období,,,,,,,,,,,,
2011-01-01,1,0,0,0,0,0,0,0,0,0,0,0
2011-02-01,0,1,0,0,0,0,0,0,0,0,0,0
2011-03-01,0,0,1,0,0,0,0,0,0,0,0,0
2011-04-01,0,0,0,1,0,0,0,0,0,0,0,0
2011-05-01,0,0,0,0,1,0,0,0,0,0,0,0


### Ted to spojime do jednoho dataframu "model_xx"
https://pandas.pydata.org/pandas-docs/stable/merging.html

In [18]:
model_R05 = pd.concat([leky_R05, M01, Q01], axis=1, join_axes=[leky_R05.index])
model_R05.head(3)

,Období,DDD_total,year,month,quater,trend,month_1,month_2,month_3,month_4,...,month_7,month_8,month_9,month_10,month_11,month_12,quater_1,quater_2,quater_3,quater_4
Období,,,,,,,,,,,,,,,,,,,,,
2011-01-01,2011-01-01,8326992.0,2011,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2011-02-01,2011-02-01,8690387.0,2011,2,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2011-03-01,2011-03-01,6590397.0,2011,3,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
results_R05 = smf.ols('DDD_total ~ trend + quater_1 + quater_2 + quater_3', data=model_R05).fit()
print(results_R05.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     22.81
Date:                Sun, 20 May 2018   Prob (F-statistic):           1.09e-12
Time:                        19:31:32   Log-Likelihood:                -1370.0
No. Observations:                  87   AIC:                             2750.
Df Residuals:                      82   BIC:                             2762.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    5.44e+06   4.59e+05     11.841      0.0

In [20]:
resultsR05 = smf.ols('DDD_total ~ trend + month_1 + month_2 + month_3 + month_4 + month_5 + month_6 + month_7 + month_8 + month_9 + month_10 + month_11', data=model_R05).fit()
print(resultsR05.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     33.31
Date:                Sun, 20 May 2018   Prob (F-statistic):           5.50e-25
Time:                        19:31:32   Log-Likelihood:                -1321.7
No. Observations:                  87   AIC:                             2669.
Df Residuals:                      74   BIC:                             2702.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.283e+06   4.24e+05     12.448      0.0